# Initialization

In [1]:
import time
from ultima.account import Account
from ultima.rest import UltimaClient, APT, USD, Buy, Sell

In [2]:
# Accounts
client = UltimaClient()
art = Account()
bud = Account()
ultima = Account(dev_nb=True)

In [3]:
# Aliases
art_ad = art.address()
bud_ad = bud.address()
u_ad = ultima.address() 

In [4]:
# Fund both with TestCoin to cover tx fees
client.mint_testcoin(art_ad)
client.mint_testcoin(bud_ad)
client.mint_testcoin(u_ad);

# Airdrop and transfers

In [5]:
# Publish an Ultima balance for Art and airdrop to him
client.publish_ultima_balances(art, u_ad)
client.airdrop_ultima_coins(ultima, art_ad, '100.3', '20000.5');

In [6]:
# Art's Regular balance
client.account_ultima_coin_balances(art_ad, u_ad)

{'APT': Decimal('100.3'), 'USD': Decimal('20000.5')}

In [7]:
# Transfer to Bud
client.publish_ultima_balances(bud, u_ad)
client.transfer_ultima_coins(art, bud_ad, u_ad, '30.1', '5000.2');

In [8]:
# Bud's balance has gone up
client.account_ultima_coin_balances(bud_ad, u_ad)

{'APT': Decimal('0'), 'USD': Decimal('5000.2')}

In [9]:
# Art's balance has gone down
client.account_ultima_coin_balances(art_ad, u_ad)

{'APT': Decimal('70.2'), 'USD': Decimal('15000.3')}

# Collateral account funding

In [10]:
# Init collateral accounts and deposit
client.init_account(art, u_ad)
client.deposit_coins(art, u_ad, '60.1', '15000.2');

In [11]:
# Art's collateral balance has gone up
client.collateral_balances(art_ad, u_ad)

{'APT': {'holdings': Decimal('60.1'), 'available': Decimal('60.1')},
 'USD': {'holdings': Decimal('15000.2'), 'available': Decimal('15000.2')}}

In [12]:
# Art's regular balance has gone down accordingly
client.account_ultima_coin_balances(art_ad, u_ad)

{'APT': Decimal('10.1'), 'USD': Decimal('0.1')}

In [13]:
# Art withdraws collateral and regular balance has increased
client.withdraw_coins(art, u_ad, '15', '4000')
client.account_ultima_coin_balances(art_ad, u_ad)

{'APT': Decimal('25.1'), 'USD': Decimal('4000.1')}

In [14]:
# Collateral account has gone down accordingly
client.collateral_balances(art_ad, u_ad)

{'APT': {'holdings': Decimal('45.1'), 'available': Decimal('45.1')},
 'USD': {'holdings': Decimal('11000.2'), 'available': Decimal('11000.2')}}

# Recording orders 

In [15]:
# Record mock orders to Art's account
r_m_o = client.record_mock_order

for id, side, price, unfilled in [
    # Various formats accepted
    (1,  Buy  ,   200    ,  '5.5' ),
    (2, 'Buy' ,  '190.5' ,  10    ),
    (3, 'Sell',  '230.0' ,  20    ),
    (4,  Buy  ,   180    ,  '7.25'),
    (5,  Sell ,   250    , '30.25')
]:
    r_m_o(ultima, art_ad, id, side, price, unfilled)
    time.sleep(.5)

In [16]:
# Orders have been recorded
client.open_orders(art_ad, u_ad)

,side,price,unfilled
id,,,
1,Buy,200.0,5.50
2,Buy,190.5,10.00
3,Sell,230.0,20.00
4,Buy,180.0,7.25
5,Sell,250.0,30.25


# Matching orders 

In [17]:
t_m_o = client.trigger_match_order

In [18]:
# Partially fill Art's order w/ id 1
t_m_o(ultima, art_ad, 1, 5, 0);

In [19]:
# Order id 1 unfilled amount has decreased
client.open_orders(art_ad, u_ad)

,side,price,unfilled
id,,,
1,Buy,200.0,0.50
2,Buy,190.5,10.00
3,Sell,230.0,20.00
4,Buy,180.0,7.25
5,Sell,250.0,30.25


In [20]:
# Collateral balances have updated
client.collateral_balances(art_ad, u_ad)

{'APT': {'holdings': Decimal('50.1'), 'available': Decimal('50.1')},
 'USD': {'holdings': Decimal('10000.2'), 'available': Decimal('10000.2')}}

In [21]:
# Completely fill Art's order w/ id 3
t_m_o(ultima, art_ad, 3, 0, 4600);

In [22]:
# Order id 3 is now gone from Art's orders
client.open_orders(art_ad, u_ad)

,side,price,unfilled
id,,,
1,Buy,200.0,0.50
2,Buy,190.5,10.00
4,Buy,180.0,7.25
5,Sell,250.0,30.25


In [23]:
# Collateral balances have updated
client.collateral_balances(art_ad, u_ad)

{'APT': {'holdings': Decimal('30.1'), 'available': Decimal('30.1')},
 'USD': {'holdings': Decimal('14600.2'), 'available': Decimal('14600.2')}}